In [ ]:
# sentiment、model、cv、feature selection、data preprocessing、encoder、evaluation、oversampling、survey

In [209]:
import pandas as pd
import numpy as np
import nltk
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from collections import Counter
pd.set_option('display.float_format', lambda x: '%.3f' % x)
sid = SentimentIntensityAnalyzer()

train = pd.read_csv('Boys_or_girls_train.csv')
test = pd.read_csv('Boys_or_girls_test_no_solution.csv')

rs =2023
#為了kaggle上傳留下的資訊
test_id = test['id']

train,test = train.drop([ 'id'], axis=1), test.drop([ 'id'], axis=1)
train['yt'], test['yt'] = train['yt'].astype(float), test['yt'].astype(float)
train['self_intro'], test['self_intro'] = train['self_intro'].astype(str), test['self_intro'].astype(str)
train['timestamp'], test['timestamp'] = pd.to_datetime(train['timestamp']), pd.to_datetime(test['timestamp'])
# train = train.sort_values('timestamp')
# test = test.sort_values('timestamp')
train.dropna(inplace=True)
# train.drop_duplicates(inplace=True)
train = train.reset_index(drop = True)

In [210]:
# #根據 timestamp 將作答時間萃取成 time_delta
# train_day1 = train.loc[(train['timestamp'] >= '2017-02-11') & (train['timestamp'] <= '2017-03-11')]
# train_day1 = train_day1.reset_index(drop=True)
# train_day1['time_delta']=0
# for i in range(len(train_day1)):
#     train_day1['time_delta'][i] = train_day1['timestamp'][i] - train_day1['timestamp'][0]
    
# train_day2 = train.loc[(train['timestamp'] >= '2017-03-11') & (train['timestamp'] <= '2017-04-07')]
# train_day2 = train_day2.reset_index(drop=True)
# train_day2['time_delta']=0
# for i in range(len(train_day2)):
#     train_day2['time_delta'][i] = train_day2['timestamp'][i] - train_day2['timestamp'][0]

# train_day3 = train.loc[(train['timestamp'] >= '2017-04-07') & (train['timestamp'] <= '2017-12-20')]
# train_day3 = train_day3.reset_index(drop=True)
# train_day3['time_delta']=0
# for i in range(len(train_day3)):
#     train_day3['time_delta'][i] = train_day3['timestamp'][i] - train_day3['timestamp'][0]
    
# train_day4 = train.loc[(train['timestamp'] >= '2017-12-20') & (train['timestamp'] <= '2017-12-21')]
# train_day4 = train_day4.reset_index(drop=True)
# train_day4['time_delta']=0
# for i in range(len(train_day4)):
#     train_day4['time_delta'][i] = train_day4['timestamp'][i] - train_day4['timestamp'][0]
    
# train_day5 = train.loc[(train['timestamp'] >= '2017-12-21') & (train['timestamp'] <= '2018-08-03')]
# train_day5 = train_day5.reset_index(drop=True)
# train_day5['time_delta']=0
# for i in range(len(train_day5)):
#     train_day5['time_delta'][i] = train_day5['timestamp'][i] - train_day5['timestamp'][0]
    
# train_day6 = train.loc[(train['timestamp'] >= '2018-08-03')]
# train_day6 = train_day6.reset_index(drop=True)
# train_day6['time_delta']=0
# for i in range(len(train_day6)):
#     train_day6['time_delta'][i] = train_day6['timestamp'][i] - train_day6['timestamp'][0]
    
# train = pd.concat([train_day1, train_day2, train_day3, train_day4, train_day5, train_day6])
# train['time_delta'] = train['time_delta'].apply(lambda x: x.total_seconds())
train = train.drop('timestamp', axis =1)
train = train.reset_index(drop=True)

In [211]:
# test_day1 = test.loc[(test['timestamp'] >= '2004-03-21') & (test['timestamp'] <= '2006-03-20')]
# test_day1 = test_day1.reset_index(drop=True)
# test_day1['time_delta']=0
# for i in range(len(test_day1)):
#     test_day1['time_delta'][i] = test_day1['timestamp'][i] - test_day1['timestamp'][0]

# test_day2 = test.loc[(test['timestamp'] >= '2006-03-20') & (test['timestamp'] <= '2006-10-18')]
# test_day2 = test_day2.reset_index(drop=True)
# test_day2['time_delta']=0
# for i in range(len(test_day2)):
#     test_day2['time_delta'][i] = test_day2['timestamp'][i] - test_day2['timestamp'][0]

# test_day3 = test.loc[(test['timestamp'] >= '2006-10-18') & (test['timestamp'] <= '2010-09-18')]
# test_day3 = test_day3.reset_index(drop=True)
# test_day3['time_delta']=0
# for i in range(len(test_day3)):
#     test_day3['time_delta'][i] = test_day3['timestamp'][i] - test_day3['timestamp'][0]

# test_day4 = test.loc[(test['timestamp'] >= '2010-09-18') & (test['timestamp'] <= '2018-03-13')]
# test_day4 = test_day4.reset_index(drop=True)
# test_day4['time_delta']=0
# for i in range(len(test_day4)):
#     test_day4['time_delta'][i] = test_day4['timestamp'][i] - test_day4['timestamp'][0]

# test_day5 = test.loc[(test['timestamp'] >= '2018-03-13') & (test['timestamp'] <= '2018-09-17')]
# test_day5 = test_day5.reset_index(drop=True)
# test_day5['time_delta']=0
# for i in range(len(test_day5)):
#     test_day5['time_delta'][i] = test_day5['timestamp'][i] - test_day5['timestamp'][0]

# test_day6 = test.loc[(test['timestamp'] >= '2018-09-17') & (test['timestamp'] <= '2019-02-22')]
# test_day6 = test_day6.reset_index(drop=True)
# test_day6['time_delta']=0
# for i in range(len(test_day6)):
#     test_day6['time_delta'][i] = test_day6['timestamp'][i] - test_day6['timestamp'][0]

# test_day7 = test.loc[(test['timestamp'] >= '2019-02-22') & (test['timestamp'] <= '2020-11-21')]
# test_day7 = test_day7.reset_index(drop=True)
# test_day7['time_delta']=0
# for i in range(len(test_day7)):
#     test_day7['time_delta'][i] = test_day7['timestamp'][i] - test_day7['timestamp'][0]
    
# test_day8 = test.loc[(test['timestamp'] >= '2020-11-21') & (test['timestamp'] <= '2021-02-25')]
# test_day8 = test_day8.reset_index(drop=True)
# test_day8['time_delta']=0
# for i in range(len(test_day8)):
#     test_day8['time_delta'][i] = test_day8['timestamp'][i] - test_day8['timestamp'][0]
    
# test_day9 = test.loc[(test['timestamp'] >= '2021-02-25') & (test['timestamp'] <= '2021-02-26')]
# test_day9 = test_day9.reset_index(drop=True)
# test_day9['time_delta']=0
# for i in range(len(test_day9)):
#     test_day9['time_delta'][i] = test_day9['timestamp'][i] - test_day9['timestamp'][0]

# test_day10 = test.loc[(test['timestamp'] >= '2021-02-26') & (test['timestamp'] <= '2021-02-27')]
# test_day10 = test_day10.reset_index(drop=True)
# test_day10['time_delta']=0
# for i in range(len(test_day10)):
#     test_day10['time_delta'][i] = test_day10['timestamp'][i] - test_day10['timestamp'][0]
    
# test_day11 = test.loc[(test['timestamp'] >= '2021-02-27')]
# test_day11 = test_day11.reset_index(drop=True)
# test_day11['time_delta']=0
# for i in range(len(test_day11)):
#     test_day11['time_delta'][i] = test_day11['timestamp'][i] - test_day11['timestamp'][0]

# test = pd.concat([test_day1, test_day2, test_day3, test_day4, test_day5, test_day6, test_day7, test_day8, test_day9, test_day10, test_day11])
# test['time_delta'] = test['time_delta'].apply(lambda x: x.total_seconds())
test = test.drop('timestamp', axis =1)
test = test.reset_index(drop=True)

In [212]:
#新增bmi
train['bmi'], test['bmi']=0,0
for i in range(len(train['height'])):
    train.loc[train.index==i,'bmi'] = train['weight'][i]/ (train['height'][i]/100)**2

for i in range(len(test['height'])):
    if test['height'][i] == 0:
        test['bmi'][i] = 0
    else:
        test.loc[test.index==i,'bmi'] = test['weight'][i]/ (test['height'][i]/100)**2 

C:\Users\user\AppData\Local\Temp\ipykernel_13336\514751759.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['bmi'][i] = 0


In [213]:
#spacy設定
nlp = spacy.load("en_core_web_lg")
nlp.add_pipe('spacytextblob')
nltk.download('punkt')
nltk.download('vader_lexicon')
train['Polarity'] = 0
train['Subjectivity'] = 0
for i in range(len(train['self_intro'])):
    text = str(train.loc[train.index==i,'self_intro'])
    doc = nlp(text)
    p = doc._.blob.polarity                            
    s = doc._.blob.subjectivity         
    train.loc[train.index==i,'Polarity'] = p
    train.loc[train.index==i,'Subjectivity'] = s     
              
test['Polarity'] = 0
test['Subjectivity'] = 0
for i in range(len(test['self_intro'])):
    text = str(test.loc[test.index==i,'self_intro'])
    doc = nlp(text)
    p = doc._.blob.polarity                            
    s = doc._.blob.subjectivity         
    test.loc[test.index==i,'Polarity'] = p
    test.loc[test.index==i,'Subjectivity'] = s   
    
#nltk
train['sentiment'], test['sentiment']=0, 0
for i in range(len(train['self_intro'])):
    score = sid.polarity_scores(train['self_intro'][i]).values()
    train.loc[train.index==i,'sentiment']= list(score)[3]
    
for i in range(len(test['self_intro'])):
    score = sid.polarity_scores(test['self_intro'][i]).values()
    test.loc[test.index==i,'sentiment']= list(score)[3]

#增加自我介紹字數欄位
train['word_count'] = 0
for i in range(len(train["self_intro"])):
    train.loc[train.index==i,'word_count'] = len(train["self_intro"][i].split())

test['word_count'] = 0
for i in range(len(test["self_intro"])):
    test.loc[test.index==i,'word_count'] = len(test["self_intro"][i].split())

#刪除self_intro
train.drop(['self_intro'], axis=1 ,inplace=True)
test.drop(['self_intro'], axis=1, inplace=True)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [214]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder()

phone_os_reshaped = train['phone_os'].values.reshape(-1, 1)
star_sign_reshaped = train['star_sign'].values.reshape(-1, 1)
phone_os_encoded = onehotencoder.fit_transform(phone_os_reshaped)
phone_os_names = onehotencoder.get_feature_names_out(['phone_os'])
star_sign_encoded = onehotencoder.fit_transform(star_sign_reshaped)
star_sign_names = onehotencoder.get_feature_names_out(['star_sign'])
dfp = pd.DataFrame(phone_os_encoded.toarray(), columns=phone_os_names)
dfs= pd.DataFrame(star_sign_encoded.toarray(),columns=star_sign_names)
train = train.drop(['star_sign','phone_os'],axis=1)
train = pd.concat([train, dfp, dfs],axis=1)
train.drop(['phone_os_JohnCena'], axis=1, inplace=True)

phone_os_reshaped = test['phone_os'].values.reshape(-1, 1)
star_sign_reshaped = test['star_sign'].values.reshape(-1, 1)
phone_os_encoded = onehotencoder.fit_transform(phone_os_reshaped)
phone_os_names = onehotencoder.get_feature_names_out(['phone_os'])
star_sign_encoded = onehotencoder.fit_transform(star_sign_reshaped)
star_sign_names = onehotencoder.get_feature_names_out(['star_sign'])
dfp = pd.DataFrame(phone_os_encoded.toarray(), columns=phone_os_names)
dfs= pd.DataFrame(star_sign_encoded.toarray(),columns=star_sign_names)

test = test.drop(['star_sign','phone_os'],axis=1)
test = pd.concat([test, dfp, dfs],axis=1)

In [215]:
from sklearn.model_selection import train_test_split
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from xgboost import XGBClassifier
import lightgbm as lgb
import smote_variants as sv
import xgboost as xgboost
from sklearn import svm
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

le = LabelEncoder()
MM = MinMaxScaler()

SvcModel=svm.SVC(random_state=rs)
LgbModel = lgb.LGBMClassifier(random_state=rs)
xgboostModel = XGBClassifier(random_state=rs)

#去掉無限值
train = train.replace([np.inf, -np.inf], np.nan)
train = train.dropna()

#切分訓練集、測試集
X = train.drop('gender',axis=1)
y = train['gender']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9,stratify=y, random_state=rs)


#需要MinMax 的欄位
mm_data = ['height', 'weight', 'iq', 'bmi','sleepiness', 'fb_friends', 'yt', 'word_count']
X_train[mm_data] = MM.fit_transform(X_train[mm_data])
X_test[mm_data] = MM.transform(X_test[mm_data])

X_train_numpy = X_train.values
y_train_numpy = y_train.values
# oversampler = sv.ProWSyn()
# X_train_oversampled, y_train_oversampled = oversampler.sample(X_train_numpy, y_train_numpy)

s_sfs = SFS(SvcModel, k_features=(1,26), forward=True, floating=False, scoring='roc_auc', cv=5)
s_sfs.fit(X_train, y_train)

l_sfs = SFS(LgbModel, k_features=(1,26), forward=True, floating=False, scoring='roc_auc', cv=5)
l_sfs.fit(X_train, y_train)

#relabel for xgboost
y_train_le = le.fit_transform(y_train)
x_sfs = SFS(xgboostModel, k_features=(1,26), forward=True, floating=False, scoring='roc_auc', cv=5)
x_sfs.fit(X_train, y_train_le )

print(f"Best score achieved: {s_sfs.k_score_}, Feature's names: {s_sfs.k_feature_names_}")
display(pd.DataFrame(s_sfs.get_metric_dict()))
print(f"Best score achieved: {l_sfs.k_score_}, Feature's names: {l_sfs.k_feature_names_}")
display(pd.DataFrame(l_sfs.get_metric_dict()))
print(f"Best score achieved: {x_sfs.k_score_}, Feature's names: {x_sfs.k_feature_names_}")
display(pd.DataFrame(x_sfs.get_metric_dict()))

Best score achieved: 0.8902400372275718, Feature's names: ('height',)


,1,2,3,4,5,6,7,8,9,10,...,17,18,19,20,21,22,23,24,25,26
feature_idx,"(0,)","(0, 13)","(0, 1, 13)","(0, 1, 13, 25)","(0, 1, 13, 23, 25)","(0, 1, 13, 15, 23, 25)","(0, 1, 13, 15, 20, 23, 25)","(0, 1, 13, 15, 17, 20, 23, 25)","(0, 1, 13, 15, 16, 17, 20, 23, 25)","(0, 1, 13, 14, 15, 16, 17, 20, 23, 25)",...,"(0, 1, 7, 9, 12, 13, 14, 15, 16, 17, 18, 20, 2...","(0, 1, 7, 8, 9, 12, 13, 14, 15, 16, 17, 18, 20...","(0, 1, 3, 7, 8, 9, 12, 13, 14, 15, 16, 17, 18,...","(0, 1, 3, 6, 7, 8, 9, 12, 13, 14, 15, 16, 17, ...","(0, 1, 3, 4, 6, 7, 8, 9, 12, 13, 14, 15, 16, 1...","(0, 1, 3, 4, 5, 6, 7, 8, 9, 12, 13, 14, 15, 16...","(0, 1, 3, 4, 5, 6, 7, 8, 9, 12, 13, 14, 15, 16...","(0, 1, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15...","(0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
cv_scores,"[0.8975069252077562, 0.8430799220272904, 0.936...","[0.8855032317636196, 0.8430799220272904, 0.936...","[0.9016620498614957, 0.850877192982456, 0.9122...","[0.9108956602031393, 0.8089668615984406, 0.893...","[0.8855032317636197, 0.7631578947368421, 0.895...","[0.8942751615881809, 0.7797270955165692, 0.827...","[0.8693444136657433, 0.7826510721247564, 0.788...","[0.8711911357340719, 0.6973684210526316, 0.774...","[0.8379501385041552, 0.6442495126705654, 0.802...","[0.7285318559556787, 0.7119883040935674, 0.796...",...,"[0.6361957525392428, 0.6306042884990254, 0.632...","[0.6472760849492152, 0.5974658869395711, 0.657...","[0.6537396121883656, 0.6101364522417154, 0.658...","[0.6666666666666666, 0.6159844054580896, 0.662...","[0.6712834718374885, 0.6130604288499025, 0.662...","[0.6712834718374884, 0.6130604288499025, 0.662...","[0.6768236380424746, 0.5828460038986354, 0.654...","[0.6694367497691597, 0.5857699805068226, 0.634...","[0.6537396121883657, 0.5750487329434697, 0.639...","[0.6472760849492152, 0.5760233918128654, 0.590..."
avg_score,0.890,0.888,0.881,0.857,0.854,0.836,0.808,0.776,0.750,0.728,...,0.629,0.647,0.648,0.654,0.654,0.654,0.649,0.643,0.640,0.637
feature_names,"(height,)","(height, phone_os_Windows Phone)","(height, weight, phone_os_Windows Phone)","(height, weight, phone_os_Windows Phone, star_...","(height, weight, phone_os_Windows Phone, star_...","(height, weight, phone_os_Windows Phone, star_...","(height, weight, phone_os_Windows Phone, star_...","(height, weight, phone_os_Windows Phone, star_...","(height, weight, phone_os_Windows Phone, star_...","(height, weight, phone_os_Windows Phone, star_...",...,"(height, weight, Polarity, sentiment, phone_os...","(height, weight, Polarity, Subjectivity, senti...","(height, weight, iq, Polarity, Subjectivity, s...","(height, weight, iq, bmi, Polarity, Subjectivi...","(height, weight, iq, fb_friends, bmi, Polarity...","(height, weight, iq, fb_friends, yt, bmi, Pola...","(height, weight, iq, fb_friends, yt, bmi, Pola...","(height, weight, iq, fb_friends, yt, bmi, Pola...","(height, weight, iq, fb_friends, yt, bmi, Pola...","(height, weight, sleepiness, iq, fb_friends, y..."
ci_bound,0.039,0.039,0.030,0.050,0.061,0.050,0.047,0.084,0.091,0.069,...,0.050,0.085,0.085,0.080,0.084,0.084,0.080,0.075,0.075,0.101
std_dev,0.030,0.030,0.023,0.039,0.048,0.039,0.036,0.065,0.071,0.054,...,0.039,0.066,0.066,0.062,0.066,0.065,0.062,0.058,0.058,0.079
std_err,0.015,0.015,0.011,0.019,0.024,0.020,0.018,0.033,0.035,0.027,...,0.019,0.033,0.033,0.031,0.033,0.032,0.031,0.029,0.029,0.039


Best score achieved: 0.9607721789854754, Feature's names: ('height', 'weight', 'sentiment', 'phone_os_Windows Phone', 'star_sign_天秤座', 'star_sign_天蠍座', 'star_sign_射手座', 'star_sign_摩羯座', 'star_sign_水瓶座', 'star_sign_牡羊座', 'star_sign_金牛座')


,1,2,3,4,5,6,7,8,9,10,...,17,18,19,20,21,22,23,24,25,26
feature_idx,"(0,)","(0, 1)","(0, 1, 19)","(0, 1, 14, 19)","(0, 1, 9, 14, 19)","(0, 1, 9, 13, 14, 19)","(0, 1, 9, 13, 14, 16, 19)","(0, 1, 9, 13, 14, 16, 19, 20)","(0, 1, 9, 13, 14, 16, 19, 20, 23)","(0, 1, 9, 13, 14, 16, 18, 19, 20, 23)",...,"(0, 1, 3, 9, 10, 13, 14, 15, 16, 18, 19, 20, 2...","(0, 1, 3, 8, 9, 10, 13, 14, 15, 16, 18, 19, 20...","(0, 1, 3, 7, 8, 9, 10, 13, 14, 15, 16, 18, 19,...","(0, 1, 3, 7, 8, 9, 10, 11, 13, 14, 15, 16, 18,...","(0, 1, 3, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17,...","(0, 1, 2, 3, 7, 8, 9, 10, 11, 13, 14, 15, 16, ...","(0, 1, 2, 3, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...","(0, 1, 2, 3, 4, 7, 8, 9, 10, 11, 12, 13, 14, 1...","(0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14...","(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
cv_scores,"[0.9067405355493998, 0.9161793372319688, 0.918...","[0.9376731301939057, 0.9502923976608186, 0.950...","[0.9496768236380425, 0.9678362573099415, 0.954...","[0.9478301015697137, 0.9727095516569201, 0.960...","[0.9418282548476454, 0.9746588693957116, 0.979...","[0.9418282548476454, 0.9746588693957116, 0.979...","[0.9418282548476454, 0.9746588693957116, 0.979...","[0.9418282548476454, 0.9746588693957116, 0.979...","[0.9418282548476454, 0.9746588693957116, 0.979...","[0.9362880886426592, 0.969785575048733, 0.9795...",...,"[0.9344413665743305, 0.9785575048732944, 0.960...","[0.9418282548476454, 0.9775828460038987, 0.969...","[0.9436749769159741, 0.9707602339181287, 0.962...","[0.9409048938134811, 0.9766081871345029, 0.960...","[0.9409048938134811, 0.9766081871345029, 0.956...","[0.9353647276084949, 0.9775828460038987, 0.959...","[0.9372114496768236, 0.9717348927875243, 0.959...","[0.9307479224376731, 0.9600389863547758, 0.956...","[0.9261311172668514, 0.9658869395711501, 0.949...","[0.9242843951985226, 0.9688109161793372, 0.947..."
avg_score,0.917,0.945,0.955,0.957,0.958,0.958,0.958,0.958,0.958,0.958,...,0.958,0.959,0.960,0.959,0.959,0.957,0.957,0.952,0.946,0.944
feature_names,"(height,)","(height, weight)","(height, weight, star_sign_水瓶座)","(height, weight, star_sign_天秤座, star_sign_水瓶座)","(height, weight, sentiment, star_sign_天秤座, sta...","(height, weight, sentiment, phone_os_Windows P...","(height, weight, sentiment, phone_os_Windows P...","(height, weight, sentiment, phone_os_Windows P...","(height, weight, sentiment, phone_os_Windows P...","(height, weight, sentiment, phone_os_Windows P...",...,"(height, weight, iq, sentiment, word_count, ph...","(height, weight, iq, Subjectivity, sentiment, ...","(height, weight, iq, Polarity, Subjectivity, s...","(height, weight, iq, Polarity, Subjectivity, s...","(height, weight, iq, Polarity, Subjectivity, s...","(height, weight, sleepiness, iq, Polarity, Sub...","(height, weight, sleepiness, iq, Polarity, Sub...","(height, weight, sleepiness, iq, fb_friends, P...","(height, weight, sleepiness, iq, fb_friends, y...","(height, weight, sleepiness, iq, fb_friends, y..."
ci_bound,0.010,0.024,0.020,0.022,0.021,0.021,0.021,0.021,0.021,0.020,...,0.021,0.023,0.016,0.021,0.021,0.023,0.023,0.019,0.019,0.020
std_dev,0.007,0.018,0.016,0.017,0.016,0.016,0.016,0.016,0.016,0.016,...,0.016,0.018,0.012,0.016,0.016,0.018,0.018,0.015,0.015,0.016
std_err,0.004,0.009,0.008,0.009,0.008,0.008,0.008,0.008,0.008,0.008,...,0.008,0.009,0.006,0.008,0.008,0.009,0.009,0.007,0.007,0.008


Best score achieved: 0.9580940655732899, Feature's names: ('height', 'weight', 'bmi', 'star_sign_摩羯座', 'star_sign_水瓶座', 'star_sign_獅子座', 'star_sign_金牛座')


,1,2,3,4,5,6,7,8,9,10,...,17,18,19,20,21,22,23,24,25,26
feature_idx,"(0,)","(0, 6)","(0, 6, 19)","(0, 1, 6, 19)","(0, 1, 6, 18, 19)","(0, 1, 6, 18, 19, 23)","(0, 1, 6, 18, 19, 21, 23)","(0, 1, 6, 13, 18, 19, 21, 23)","(0, 1, 6, 13, 18, 19, 20, 21, 23)","(0, 1, 6, 13, 16, 18, 19, 20, 21, 23)",...,"(0, 1, 4, 6, 10, 11, 12, 13, 14, 16, 17, 18, 1...","(0, 1, 4, 6, 10, 11, 12, 13, 14, 16, 17, 18, 1...","(0, 1, 4, 6, 10, 11, 12, 13, 14, 16, 17, 18, 1...","(0, 1, 4, 6, 10, 11, 12, 13, 14, 15, 16, 17, 1...","(0, 1, 4, 6, 9, 10, 11, 12, 13, 14, 15, 16, 17...","(0, 1, 4, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16,...","(0, 1, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, ...","(0, 1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...","(0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
cv_scores,"[0.9044321329639889, 0.9152046783625731, 0.923...","[0.9270544783010156, 0.969785575048733, 0.9541...","[0.9325946445060018, 0.9678362573099415, 0.957...","[0.943213296398892, 0.9883040935672515, 0.9561...","[0.9385964912280702, 0.9873294346978557, 0.961...","[0.9385964912280702, 0.9883040935672515, 0.961...","[0.943213296398892, 0.9883040935672515, 0.9585...","[0.943213296398892, 0.9883040935672515, 0.9585...","[0.943213296398892, 0.9873294346978558, 0.9585...","[0.943213296398892, 0.9892787524366471, 0.9585...",...,"[0.9353647276084949, 0.9824561403508772, 0.955...","[0.9307479224376731, 0.9853801169590644, 0.955...","[0.9390581717451523, 0.9873294346978557, 0.946...","[0.9390581717451523, 0.9824561403508772, 0.946...","[0.9279778393351801, 0.9658869395711501, 0.959...","[0.9187442289935365, 0.9688109161793372, 0.954...","[0.9316712834718375, 0.9746588693957114, 0.944...","[0.9307479224376731, 0.9736842105263157, 0.946...","[0.9344413665743305, 0.9658869395711501, 0.948...","[0.9205909510618652, 0.9678362573099415, 0.950..."
avg_score,0.915,0.950,0.954,0.955,0.957,0.958,0.958,0.958,0.957,0.957,...,0.957,0.957,0.957,0.954,0.953,0.951,0.951,0.951,0.948,0.946
feature_names,"(height,)","(height, bmi)","(height, bmi, star_sign_水瓶座)","(height, weight, bmi, star_sign_水瓶座)","(height, weight, bmi, star_sign_摩羯座, star_sign...","(height, weight, bmi, star_sign_摩羯座, star_sign...","(height, weight, bmi, star_sign_摩羯座, star_sign...","(height, weight, bmi, phone_os_Windows Phone, ...","(height, weight, bmi, phone_os_Windows Phone, ...","(height, weight, bmi, phone_os_Windows Phone, ...",...,"(height, weight, fb_friends, bmi, word_count, ...","(height, weight, fb_friends, bmi, word_count, ...","(height, weight, fb_friends, bmi, word_count, ...","(height, weight, fb_friends, bmi, word_count, ...","(height, weight, fb_friends, bmi, sentiment, w...","(height, weight, fb_friends, bmi, Subjectivity...","(height, weight, fb_friends, yt, bmi, Subjecti...","(height, weight, fb_friends, yt, bmi, Polarity...","(height, weight, sleepiness, fb_friends, yt, b...","(height, weight, sleepiness, iq, fb_friends, y..."
ci_bound,0.012,0.024,0.023,0.026,0.026,0.025,0.024,0.024,0.024,0.025,...,0.023,0.026,0.024,0.022,0.017,0.022,0.018,0.019,0.013,0.020
std_dev,0.010,0.019,0.018,0.020,0.020,0.020,0.019,0.019,0.019,0.019,...,0.018,0.020,0.019,0.017,0.013,0.017,0.014,0.015,0.010,0.016
std_err,0.005,0.009,0.009,0.010,0.010,0.010,0.009,0.009,0.009,0.010,...,0.009,0.010,0.009,0.009,0.007,0.009,0.007,0.007,0.005,0.008


In [216]:
import lightgbm as lgb
import smote_variants as sv
import xgboost as xgboost
from sklearn import svm
from sklearn.metrics import (auc, classification_report, roc_auc_score,roc_curve)
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from xgboost import XGBClassifier
from sklearn import metrics
from  statistics import mean
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

le = LabelEncoder()
MM = MinMaxScaler()

X_train = X_train.replace([np.inf, -np.inf], np.nan)
X_train = X_train.dropna()


features = X_train
name = features.columns.values
label_y = y_train
feature = features.to_numpy()
label = label_y.to_numpy().astype('int')

In [217]:
#Kfold cv 資料
s_auc=[]
s_acc=[]
s_ans=[]
s_sfs_features =[]
l_auc=[]
l_acc=[]
l_ans=[]
l_sfs_features =[]
x_auc=[]
x_acc=[]
x_ans=[]
x_sfs_features =[]

kfold = StratifiedKFold(n_splits=5, random_state=rs,shuffle=True).split(feature, label)

for n, (ttrain, valid) in enumerate(kfold):
    
    x_train = pd.DataFrame(feature[ttrain])
    x_train.columns = name
    y_train = label[ttrain]
    y_train = le.fit_transform(y_train)
    x_valid = pd.DataFrame(feature[valid])
    x_valid.columns = name
    y_valid = label[valid]
    y_valid = le.transform(y_valid)
    
    # #MinMax
    # x_train[mm_data] = MM.fit_transform(X_train[mm_data])
    # x_valid[mm_data] = MM.transform(x_valid[mm_data])
    
    
    
    #Oversample
    oversampler = sv.ProWSyn(random_state=rs)
    x_os, y_os = oversampler.sample(x_train.to_numpy(), y_train.reshape(-1,))
    
    #SVM
    # s_sfs = SFS(SvcModel,
    #         k_features=(1,27),
    #         forward=True, 
    #         floating=False,
    #         scoring='accuracy',
    #         cv=5)
    # s_sfs.fit(X_os, y_os)
    # s_sfs_features.append(s_sfs_feature)
    s_sfs_feature=[]
    for i in s_sfs.k_feature_names_:
        s_sfs_feature.append(i)  
    x_os_s = pd.DataFrame(x_os, columns=name)
    x_os_s = x_os_s[s_sfs_feature]
    x_valid_s = x_valid[s_sfs_feature]
    X_test_s = X_test[s_sfs_feature]
    
    SvcModel.fit(x_os_s, y_os)
    s_pred=SvcModel.predict(x_valid_s)
    fpr, tpr, thresholds = metrics.roc_curve(y_valid.astype(int),s_pred, pos_label=1)
    s_acc.append(metrics.accuracy_score(s_pred,y_valid.reshape(-1,).astype(int)))
    s_auc.append(metrics.auc(fpr, tpr))
    # ans=SvcModel.predict(X_test)
    # s_ans.append(ans)
    
    
    #Lgbm
    # l_sfs = SFS(LgbModel,
    #         k_features=(1,27),
    #         forward=True, 
    #         floating=False,
    #         scoring='accuracy',
    #         cv=5)
    # l_sfs.fit(x_os, y_os)
    l_sfs_feature=[]
    for i in l_sfs.k_feature_names_:
        l_sfs_feature.append(i)
    # l_sfs_features.append(l_sfs_feature)     
    x_os_l = pd.DataFrame(x_os, columns=name)
    x_os_l = x_os_l[l_sfs_feature]
    x_valid_s = x_valid[l_sfs_feature]
    
    LgbModel.fit(x_os_l, y_os)
    l_pred=LgbModel.predict(x_valid_s)
    fpr, tpr, thresholds = metrics.roc_curve(y_valid.astype(int),l_pred, pos_label=1)
    l_acc.append(metrics.accuracy_score(l_pred,y_valid.reshape(-1,).astype(int)))
    l_auc.append(metrics.auc(fpr, tpr))
    # ans=LgbModel.predict(X_test)
    # l_ans.append(ans)


    #xgboost   
    # x_sfs = SFS(xgboostModel,
    #         k_features=(1,27),
    #         forward=True, 
    #         floating=False,
    #         scoring='accuracy',
    #         cv=5)
    # x_sfs.fit(x_os, y_os)
    x_sfs_feature=[]
    for i in x_sfs.k_feature_names_:
        x_sfs_feature.append(i)    
    # x_sfs_features.append(x_sfs_feature)
    x_os_x = pd.DataFrame(x_os, columns=name)
    x_os_x = x_os_x[x_sfs_feature]
    x_valid_x = x_valid[x_sfs_feature]

    xgboostModel.fit(x_os_x, y_os)
    x_pred=xgboostModel.predict(x_valid_x)
    fpr, tpr, thresholds = metrics.roc_curve(y_valid.astype(int),x_pred, pos_label=1)
    x_acc.append(metrics.accuracy_score(x_pred,y_valid.reshape(-1,).astype(int)))
    x_auc.append(metrics.auc(fpr, tpr))
    # ans=xgboostModel.predict(X_test_x)
    # x_ans.append(ans)
    
# print("SVM's features: ",s_sfs_features)    
print("SVM\nacc:{}\nauc:{}\n".format(mean(s_acc),mean(s_auc)))  
# print("Lgbm's features: ",l_sfs_features)   
print("Lgbm\nacc:{}\nauc:{}\n".format(mean(l_acc),mean(l_auc)))
# print("Xgboost's features: ",x_sfs_features)    
print("Xgboost\nacc:{}\nauc:{}\n".format(mean(x_acc),mean(x_auc)))

2023-03-31 22:24:35,196:INFO:ProWSyn: Running sampling via ('ProWSyn', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'L': 5, 'theta': 1.0, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': 2023, 'class_name': 'ProWSyn'}")
2023-03-31 22:24:35,196:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2023-03-31 22:24:35,197:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2023-03-31 22:24:35,199:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2023-03-31 22:24:35,199:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2023-03-31 22:24:35,204:INFO:ProWSyn: simplex sampling with n_dim 2
2023-03-31 22:24:35,206:INFO:ProWSyn: The simplex order was updated from 2 to 1
2023-03-31 22:24:35,206:INFO:ProWSyn: simplex sampling with n_dim 1
2023-03-31 22:24:35,313:INFO:ProWSyn: Running sampling via ('ProWSyn', "{'proport

SVM
acc:0.37533333333333335
auc:0.5718045112781955

Lgbm
acc:0.9148421052631579
auc:0.9077380952380952

Xgboost
acc:0.9201754385964912
auc:0.9007518796992481



In [218]:
test_s_auc=[]
test_s_acc=[]
test_s_ans=[]
X_test_for_s = X_test[s_sfs_feature]
test_s_pred=SvcModel.predict(X_test_for_s)
for i in range(len(test_s_pred)):
    test_s_pred[i] += 1  
fpr, tpr, thresholds = metrics.roc_curve(y_test.astype(int),test_s_pred, pos_label=2)
test_s_acc.append(metrics.accuracy_score(test_s_pred,y_test))
test_s_auc.append(metrics.auc(fpr, tpr))
print("SvcModel test \nacc:{}\nauc:{}\n".format(mean(test_s_acc),mean(test_s_auc)))

SvcModel test 
acc:0.23809523809523808
auc:0.5



In [219]:
test_l_auc=[]
test_l_acc=[]
test_l_ans=[]
X_test_for_l = X_test[l_sfs_feature]
test_l_pred=LgbModel.predict(X_test_for_l)
for i in range(len(test_l_pred)):
    test_l_pred[i] += 1  
fpr, tpr, thresholds = metrics.roc_curve(y_test.astype(int),test_l_pred, pos_label=2)
test_l_acc.append(metrics.accuracy_score(test_l_pred,y_test))
test_l_auc.append(metrics.auc(fpr, tpr))
print("LgbModel test \nacc:{}\nauc:{}\n".format(mean(test_l_acc),mean(test_l_auc)))

LgbModel test 
acc:0.8809523809523809
auc:0.8875



In [220]:
test_x_auc=[]
test_x_acc=[]
test_x_ans=[]
X_test_for_x = X_test[x_sfs_feature]
test_x_pred=xgboostModel.predict(X_test_for_x)
#(0,1)->(1,2)
for i in range(len(test_x_pred)):
    test_x_pred[i] += 1  
fpr, tpr, thresholds = metrics.roc_curve(y_test.astype(int),test_x_pred, pos_label=2)
test_x_acc.append(metrics.accuracy_score(test_x_pred,y_test))
test_x_auc.append(metrics.auc(fpr, tpr))
print("Xgboost test \nacc:{}\nauc:{}\n".format(mean(test_x_acc),mean(test_x_auc)))

Xgboost test 
acc:0.8333333333333334
auc:0.821875



In [221]:
#X_final = test[s_sfs_feature]
#final_ans=SvcModel.predict(X_final)

X_final = test[l_sfs_feature]
final_ans=LgbModel.predict(X_final)

#X_final = test[x_sfs_feature]
#final_ans=xgboostModel.predict(X_final)

for i in range(len(final_ans)):
    final_ans[i] += 1  
final_ans
output = pd.DataFrame(test_id)
output.insert(1, column="gender", value=final_ans)
output.to_csv("a2.csv", index=False)
Counter(output['gender'])

Counter({1: 206, 2: 37})

In [222]:
y_test = y_test.to_frame()
Counter(y_test['gender'])

Counter({1: 32, 2: 10})